In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [3]:
# Importiamo le librerie necessarie: transforms da torchvision per le trasformazioni delle immagini
import torchvision.transforms as transforms
import torchvision
import torch

# Definiamo le trasformazioni da applicare al dataset (normalizzare le immagini e convertirle in tensori)
# transforms.Compose permette di concatenare più trasformazioni
transform = transforms.Compose(
    [
        # Convertiamo le immagini in tensori (vettori multidimensionali) con valori normalizzati tra 0 e 1
        transforms.ToTensor(), 
        
        # Normalizziamo le immagini per avere valori medi centrati a 0.5 con deviazione standard 0.5
        # Ogni canale dell'immagine (rosso, verde, blu) viene normalizzato separatamente
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
    ])

# Scarica e carica il dataset CIFAR-10 per il training, salvando i dati nella cartella './data'
# 'train=True' indica che stiamo scaricando il dataset di addestramento
# 'download=True' permette di scaricare i dati se non sono già presenti nella directory specificata
# 'transform=transform' applica le trasformazioni definite sopra (conversione in tensori e normalizzazione)
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# Utilizziamo DataLoader per caricare il dataset in mini-batch per il training
# 'batch_size=100' significa che ogni batch sarà composto da 100 immagini
# 'shuffle=True' mescola i dati ad ogni epoca, utile per evitare correlazioni durante l'addestramento
# 'num_workers=2' indica che due thread verranno utilizzati per caricare i dati in parallelo, velocizzando il caricamento
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

# Scarica e carica il dataset CIFAR-10 per il test, questa volta 'train=False' indica che è il dataset di test
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Anche per il dataset di test utilizziamo DataLoader per caricare i dati
# 'batch_size=100' indica che i batch di test avranno 100 immagini ciascuno
# 'shuffle=False' poiché nel test non è necessario mescolare i dati
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# Definiamo le classi presenti nel dataset CIFAR-10
# CIFAR-10 è composto da 10 categorie di oggetti differenti (aerei, auto, uccelli, gatti, cervi, cani, rane, cavalli, navi e camion)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')


100%|██████████| 170498071/170498071 [00:18<00:00, 9388501.00it/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:


# Definiamo una rete neurale convoluzionale (CNN) che eredita da nn.Module
class CNN(nn.Module):
    def __init__(self):
        # Inizializziamo la classe base nn.Module
        super(CNN, self).__init__()

        # Primo strato convoluzionale: prende in input 3 canali (immagine a colori RGB), 
        # genera 32 canali (filtri), kernel di dimensione 3x3 e padding=1 per mantenere la dimensione dell'immagine
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        
        # Secondo strato convoluzionale: prende 32 canali in input e genera 64 canali, 
        # kernel di dimensione 3x3 e padding=1 per mantenere la dimensione
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)

        # Strato di pooling (max pooling): riduce la dimensione dell'immagine di un fattore 2x2
        self.pool = nn.MaxPool2d(2, 2)

        # Primo strato completamente connesso (fully connected): input = 64 canali di feature map di dimensione 8x8,
        # output = 512 neuroni. Viene utilizzato dopo la convoluzione e il pooling
        self.fc1 = nn.Linear(64 * 8 * 8, 512)

        # Secondo strato completamente connesso (fully connected): input = 512 neuroni, output = 10 neuroni 
        # (corrispondenti alle 10 classi del CIFAR-10)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        # Applicazione della prima convoluzione, seguita da funzione di attivazione ReLU e pooling
        x = self.pool(torch.relu(self.conv1(x)))
        
        # Applicazione della seconda convoluzione, seguita da funzione di attivazione ReLU e pooling
        x = self.pool(torch.relu(self.conv2(x)))

        # "Appiattimento" del tensore: converte la feature map 2D in un vettore 1D, necessario per i layer fully connected.
        # view(-1, 64 * 8 * 8) ridimensiona il tensore in modo che il primo asse (batch size) rimanga invariato,
        # mentre tutte le altre dimensioni vengono compattate in un unico vettore.
        x = x.view(-1, 64 * 8 * 8)

        # Applicazione del primo strato fully connected con funzione di attivazione ReLU
        x = torch.relu(self.fc1(x))

        # Applicazione del secondo strato fully connected, che genera le previsioni finali (logits) per le 10 classi
        x = self.fc2(x)
        
        # Restituisce l'output finale (senza softmax poiché il calcolo della loss la richiede separatamente)
        return x


In [7]:
# Creiamo un'istanza del modello CNN precedentemente definito
# 'model' è ora la nostra rete convoluzionale che utilizzeremo per l'addestramento e la predizione
model = CNN()

# Definiamo la funzione di perdita (loss function) per il compito di classificazione.
# Utilizziamo 'CrossEntropyLoss', che è comunemente usata nei problemi di classificazione multi-classe.
# Questa funzione calcola la differenza tra le previsioni del modello (logits) e le etichette reali (classi corrette).
criterion = nn.CrossEntropyLoss()

# Definiamo l'ottimizzatore che sarà usato per aggiornare i pesi della rete durante l'addestramento.
# Qui stiamo utilizzando 'Adam', un ottimizzatore avanzato che combina i vantaggi di Adagrad e RMSprop.
# Adam adatta dinamicamente il tasso di apprendimento per ogni parametro, rendendolo molto efficace e robusto.
# 'model.parameters()' passa i parametri del modello (pesi e bias) che devono essere ottimizzati.
# 'lr=0.001' imposta il tasso di apprendimento iniziale a 0.001, che controlla la dimensione del passo
# che l'ottimizzatore farà ad ogni aggiornamento dei pesi.
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [9]:
# Addestramento del modello CNN
# 'epoch' indica una passata completa sul dataset di addestramento
# In questo ciclo, eseguiamo l'addestramento per 10 epoche (iterazioni sull'intero dataset)
for epoch in range(10):  # ciclo sul dataset più volte
    running_loss = 0.0  # Variabile per tracciare la perdita media durante l'epoca

    # Enumeriamo i dati presenti nel 'trainloader', che contiene i batch di immagini e le rispettive etichette
    # 'i' è l'indice del batch, 'data' è una lista che contiene gli input (immagini) e le etichette (classi corrette)
    for i, data in enumerate(trainloader, 0):
        # Otteniamo gli input e le etichette dal batch corrente. 'data' è una lista [inputs, labels]
        inputs, labels = data

        # Azzeriamo i gradienti dei parametri del modello (pesi e bias)
        # Questo è necessario perché PyTorch accumula i gradienti ad ogni iterazione,
        # quindi dobbiamo resettarli prima di ogni passo di ottimizzazione.
        optimizer.zero_grad()

        # **Forward pass**: Passiamo gli input attraverso il modello per ottenere le previsioni (output)
        outputs = model(inputs)

        # Calcoliamo la perdita (loss) confrontando le previsioni del modello (outputs) con le etichette reali (labels)
        # La funzione di perdita 'criterion' (CrossEntropyLoss) misura quanto le previsioni sono lontane dalle etichette corrette.
        loss = criterion(outputs, labels)

        # **Backward pass**: Calcoliamo i gradienti della perdita rispetto ai parametri del modello
        # Questo passaggio calcola il gradiente per ciascun parametro in base all'errore commesso.
        loss.backward()

        # **Ottimizzazione**: Aggiorniamo i pesi del modello utilizzando l'ottimizzatore Adam,
        # che utilizza i gradienti calcolati nel backward pass per modificare i parametri e ridurre la perdita.
        optimizer.step()

        # Tracciamo la perdita accumulata per ogni batch per poter monitorare l'addestramento
        running_loss += loss.item()

        # Ogni 100 batch, stampiamo le statistiche (ad esempio, la perdita media per gli ultimi 100 batch)
        if i % 100 == 99:    # Stampiamo ogni 100 batch
            # La perdita media per ogni 100 batch viene stampata su schermo per monitorare il progresso
            print(f'Epoch {epoch + 1}, Batch {i + 1}: Loss {running_loss / 100:.3f}')
            running_loss = 0.0  # Reset della variabile running_loss per il prossimo gruppo di batch

# Dopo aver completato tutte le epoche, l'addestramento è finito
print('Training completed.')


Epoch 1, Batch 100: Loss 1.768
Epoch 1, Batch 200: Loss 1.396
Epoch 1, Batch 300: Loss 1.248
Epoch 1, Batch 400: Loss 1.142
Epoch 1, Batch 500: Loss 1.098
Epoch 2, Batch 100: Loss 0.975
Epoch 2, Batch 200: Loss 0.951
Epoch 2, Batch 300: Loss 0.920
Epoch 2, Batch 400: Loss 0.893
Epoch 2, Batch 500: Loss 0.866
Epoch 3, Batch 100: Loss 0.761
Epoch 3, Batch 200: Loss 0.755
Epoch 3, Batch 300: Loss 0.745
Epoch 3, Batch 400: Loss 0.749
Epoch 3, Batch 500: Loss 0.737
Epoch 4, Batch 100: Loss 0.598
Epoch 4, Batch 200: Loss 0.623
Epoch 4, Batch 300: Loss 0.615
Epoch 4, Batch 400: Loss 0.597
Epoch 4, Batch 500: Loss 0.608
Epoch 5, Batch 100: Loss 0.442
Epoch 5, Batch 200: Loss 0.452
Epoch 5, Batch 300: Loss 0.472
Epoch 5, Batch 400: Loss 0.485
Epoch 5, Batch 500: Loss 0.501
Epoch 6, Batch 100: Loss 0.320
Epoch 6, Batch 200: Loss 0.338
Epoch 6, Batch 300: Loss 0.333
Epoch 6, Batch 400: Loss 0.364
Epoch 6, Batch 500: Loss 0.364
Epoch 7, Batch 100: Loss 0.208
Epoch 7, Batch 200: Loss 0.207
Epoch 7,

In [11]:
# Inizializziamo le variabili per tenere traccia del numero totale di predizioni corrette e del numero totale di immagini
correct = 0  # Numero di predizioni corrette
total = 0    # Numero totale di immagini testate

# Disabilitiamo il calcolo del gradiente durante il testing.
# Questo è importante perché durante il testing non aggiorniamo i pesi, quindi non è necessario calcolare i gradienti,
# risparmiando memoria e velocizzando il processo.
with torch.no_grad():
    # Iteriamo attraverso i batch del test loader (testloader), che contiene i dati di test
    for data in testloader:
        # Estraiamo le immagini e le etichette reali (labels) dal batch corrente
        images, labels = data

        # Passiamo le immagini attraverso il modello per ottenere le previsioni
        outputs = model(images)

        # Otteniamo le predizioni dal modello.
        # torch.max(outputs, 1) restituisce il valore massimo e l'indice della classe con il valore più alto
        # lungo la dimensione 1 (le classi). Non ci interessa il valore massimo in sé, ma solo l'indice (classe predetta).
        _, predicted = torch.max(outputs, 1)

        # Aggiorniamo il conteggio del totale delle immagini testate
        total += labels.size(0)  # labels.size(0) restituisce il numero di immagini nel batch (batch size)

        # Sommiamo il numero di predizioni corrette.
        # (predicted == labels) restituisce un tensore booleano con True per ogni predizione corretta,
        # sommiamo quindi tutti i True (che valgono 1) usando .sum().item() per ottenere il numero totale di corrette.
        correct += (predicted == labels).sum().item()

# Calcoliamo e stampiamo l'accuratezza del modello sulle immagini di test.
# L'accuratezza è la percentuale di predizioni corrette rispetto al totale delle immagini testate.
# Viene moltiplicata per 100 per ottenere la percentuale.
print(f'Accuracy of the network on the 10,000 test images: {100 * correct / total:.2f}%')


Accuracy of the network on the 10,000 test images: 72.57%


In [13]:
# Save the trained model
torch.save(model.state_dict(), 'cnn_cifar10.pth')


In [27]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# CIFAR-10 class names
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

# Load the saved model
model = CNN()
model.load_state_dict(torch.load('cnn_cifar10.pth'))
model.eval()

# Preprocess the custom image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    img = Image.open(image_path)
    img = transform(img)
    img = img.unsqueeze(0)
    
    return img

# Make predictions
def predict_image(model, image_path):
    img = preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(img)
        _, predicted = torch.max(outputs, 1)
    predicted_class = classes[predicted[0]]
    print(f'Predicted Class: {predicted_class}')
    return predicted_class

# Example usage: Upload and predict a custom image
image_path = 'images.jpeg'
predict_image(model, image_path)

def predict_confidence(model, image_path):
    img = preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(img)
        probabilities = torch.softmax(outputs, dim=1)
        return {classes[i]: float(probabilities[0][i]) * 100 for i in range(len(classes))}

print(predict_confidence(model, image_path))


Predicted Class: plane
{'plane': 64.62109684944153, 'car': 0.0011086426638939884, 'bird': 34.6878707408905, 'cat': 4.2893901763818576e-07, 'deer': 0.002944551124528516, 'dog': 0.023453608446288854, 'frog': 0.66287643276155, 'horse': 0.0006325258709694026, 'ship': 1.1265018429185147e-05, 'truck': 5.169644623492786e-06}


C:\Users\mbagn\AppData\Local\Temp\ipykernel_40052\3224306955.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('cnn_cifar10.pth'))


In [ ]:
pip install fastapi uvicorn pillow torch torchvision


In [17]:
# Importiamo le librerie necessarie da FastAPI per la gestione del server web e delle risposte
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse

# Importiamo PyTorch per gestire il modello, la rete neurale e il preprocessing
import torch
import torch.nn as nn
import torchvision.transforms as transforms

# Importiamo PIL per la gestione delle immagini
from PIL import Image
import io

# Definiamo l'architettura della CNN (rete neurale convoluzionale)
# Questa è una semplice rete che utilizziamo per fare previsioni su immagini del dataset CIFAR-10
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Primo strato convoluzionale: 3 canali in input (immagini RGB) e 32 canali in output (filtri)
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        # Secondo strato convoluzionale: 32 canali in input e 64 in output
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # Strato di pooling per ridurre la dimensione (2x2)
        self.pool = nn.MaxPool2d(2, 2)
        # Primo strato completamente connesso (fully connected), dimensione 64*8*8 = 512 neuroni
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        # Secondo strato completamente connesso che riduce a 10 classi (per CIFAR-10)
        self.fc2 = nn.Linear(512, 10)

    # Definizione del forward pass: specifica come i dati passano attraverso la rete
    def forward(self, x):
        # Primo strato convoluzionale, seguito da ReLU e Max Pooling
        x = self.pool(torch.relu(self.conv1(x)))
        # Secondo strato convoluzionale, seguito da ReLU e Max Pooling
        x = self.pool(torch.relu(self.conv2(x)))
        # Appiattiamo l'output convoluzionale in un vettore 1D per passare ai fully connected layer
        x = x.view(-1, 64 * 8 * 8)
        # Primo fully connected layer seguito da ReLU
        x = torch.relu(self.fc1(x))
        # Secondo fully connected layer che restituisce l'output finale (logits per 10 classi)
        x = self.fc2(x)
        return x

# Definiamo i nomi delle classi del dataset CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

# Inizializziamo l'applicazione FastAPI
app = FastAPI()

# Carichiamo il modello pre-addestrato salvato su file (cnn_cifar10.pth)
# 'model.load_state_dict' carica i pesi nel modello definito sopra
model = CNN()
model.load_state_dict(torch.load('cnn_cifar10.pth'))
model.eval()  # Impostiamo il modello in modalità "evaluation" per disabilitare il calcolo dei gradienti

# Funzione per il preprocessing dell'immagine prima di passarla al modello
# Questa funzione applica le trasformazioni necessarie all'immagine caricata dall'utente
def preprocess_image(image_data):
    # Definiamo una sequenza di trasformazioni per ridimensionare l'immagine, convertirla in tensore e normalizzarla
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # Ridimensioniamo l'immagine a 32x32 pixel (dimensione richiesta per CIFAR-10)
        transforms.ToTensor(),  # Convertiamo l'immagine in un tensore
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizziamo i canali R, G, B tra [-1, 1]
    ])
    
    # Apriamo l'immagine da un flusso di byte in memoria (usiamo io.BytesIO per gestire i dati binari)
    image = Image.open(io.BytesIO(image_data))
    
    # Applichiamo le trasformazioni definite e aggiungiamo una dimensione batch (usiamo unsqueeze(0))
    image = transform(image).unsqueeze(0)
    return image  # Restituiamo l'immagine preprocessata come tensore

# Definiamo la rotta POST /predict per caricare un'immagine e fare la predizione
# FastAPI gestisce il caricamento dell'immagine tramite il parametro 'file' usando 'UploadFile'
@app.post("/predict/")
async def predict_image(file: UploadFile = File(...)):
    # Leggiamo il contenuto del file caricato dall'utente
    image_data = await file.read()

    # Preprocessiamo l'immagine (ridimensionamento, conversione a tensore, normalizzazione)
    img_tensor = preprocess_image(image_data)

    # Disabilitiamo il calcolo dei gradienti (torch.no_grad) poiché siamo in modalità di inferenza (predizione)
    with torch.no_grad():
        # Passiamo l'immagine preprocessata attraverso il modello per ottenere i logits (output grezzo)
        outputs = model(img_tensor)
        
        # torch.max ottiene l'indice della classe con il punteggio più alto
        # Questo indice rappresenta la classe predetta dal modello
        _, predicted = torch.max(outputs, 1)

    # Convertiamo l'indice della classe predetta nel nome della classe (ad esempio, 'dog', 'car', etc.)
    predicted_class = classes[predicted[0]]
    
    # Restituiamo la classe predetta come risposta JSON al client
    return JSONResponse(content={"predicted_class": predicted_class})


ModuleNotFoundError: No module named 'fastapi'

In [ ]:
python -m uvicorn app:app --reload
http://127.0.0.1:8000/docs

In [ ]:
https://www.kaggle.com/datasets/zalando-research/fashionmnist